In [1]:
import numpy as np
import sys
import math
sys.path.append('./Slope_SVM')

from Our_methods import use_FOM_CGCP, use_FOM
from simulate_data_classification import simulate_data_classification
from CVX_L0_Slope import CVX_L0_Slope


In [2]:

## generate data 
## 'trash.txt' is a temperory txt file to save some of the output

N, P = 100, 10000
type_Sigma, k0, rho, tau_SNR = 2, 10, 0.1, 1
seed_X = 134
f = open('trash.txt', 'w')
X_train, _, _, y_train, _, _ = simulate_data_classification(type_Sigma, N, P, k0, rho, tau_SNR, seed_X, f, create_test=False)

lam_max = np.max(np.sum( np.abs(X_train), axis=0))
lam = 0.01*lam_max

## lambda_type = 0: Not all lambda are distinct
## lambda_type = 1: All lambda are distinct
lambda_type = 0

if lambda_type:
    K_Slope    = 0 
    lambda_arr = np.array([math.sqrt(math.log(2.*P/(j+1))) for j in range(P)])
else:
    K_Slope    = 10
    lambda_arr = np.array([2 for _ in range(K_Slope)] + [1 for _ in range(P-K_Slope)])




DATA CREATED for N=100, P=10000, k0=10 Rho=0.1 Sigma=2 Seed_X=134


In [3]:
## Use column generation and constraint generation LP inicialized with first order method (FO-CLG). See the paper for more details.

obj, time_total, time_CG, beta, beta0 = use_FOM_CGCP(X_train, y_train, lam, lambda_type, tol=0.01)

Number of iterations: 200
Convergence: 0.007
Len support proximal: 77
Time proximal: 1.572

## Number of iterations outer loop: 1
## Total time smoothing for hinge: 1.572
Size of model: [100, 77]
Using license file /home/software/gurobi/gurobi.lic
Set parameter TokenServer to value engaging-license-001
<gurobipy.ParamClass: Parameter changes: TokenServer=engaging-license-001, OutputFlag=0>
Round 1
Number of columns added: 10
Round 2
57.854653507745006 57.266954833722316
Number of columns added: 10
Round 3
Number of columns added: 10
Round 4
57.80594818990359 56.975628254770044
Number of columns added: 10
Round 5
57.62392150783981 56.95970948332713
Number of columns added: 10
Round 6
57.692154475611 56.92608745781753
Number of columns added: 7
Round 7
57.69000757405096 56.898353142140074
Number of columns added: 9
Round 8
57.81674454593543 56.83322909543013
Number of columns added: 1
Round 9
57.56787905041199 56.94298021943587
Number of columns added: 1
Round 10
57.82971861210242 56.935

In [4]:

# use Gurobi
# Note that we use Gurobi only when lambda_type = 0
model = "gurobi"
obj, runtime, beta, beta0 = CVX_L0_Slope(X_train, y_train, lam, K_Slope, f, model)


Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 60200 rows, 40102 columns and 1120300 nonzeros
Model fingerprint: 0x68f74d8b
Coefficient statistics:
  Matrix range     [5e-07, 1e+00]
  Objective range  [9e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 30100 rows and 10000 columns
Presolve time: 1.07s
Presolved: 30100 rows, 30102 columns, 1080200 nonzeros

Ordering time: 0.03s

Barrier statistics:
 Dense cols : 1
 Free vars  : 10002
 AA' NZ     : 3.055e+06
 Factor NZ  : 3.096e+06 (roughly 50 MBytes of memory)
 Factor Ops : 3.189e+08 (less than 1 second per iteration)
 Threads    : 26

                  Objective                Residual
Iter       Primal          Du

In [30]:
# use ECOS
# Note that we use ECOS only when lambda_type = 0
model = "ecos"
obj, runtime, beta, beta0 = CVX_L0_Slope(X_train, y_train, lam, K_Slope, f, model)



ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +3.732e+00  +1.110e+02  +7e+04  1e+00  4e+01  1e+00  1e+00    ---    ---    1  2  - |  -  - 
 1  -7.022e+00  +5.881e+00  +6e+04  4e-02  2e+00  6e+00  1e+00  0.6487  8e-01   2  1  1 |  0  0
 2  -6.876e-01  +6.359e+00  +3e+04  3e-02  1e+00  3e+00  6e-01  0.5580  3e-01   2  1  1 |  0  0
 3  +4.924e-01  +5.921e+00  +3e+04  3e-02  1e+00  2e+00  5e-01  0.3298  4e-01   2  1  1 |  0  0
 4  +2.515e+00  +4.212e+00  +1e+04  8e-03  4e-01  6e-01  2e-01  0.8053  3e-01   2  1  1 |  0  0
 5  +2.787e+00  +3.623e+00  +6e+03  4e-03  2e-01  2e-01  1e-01  0.5657  2e-01   1  1  1 |  0  0
 6  +2.883e+00  +3.534e+00  +5e+03  3e-03  2e-01  2e-01  9e-02  0.3791  6e-01   1  1  1 |  0  0
 7  +2.968e+00  +3.282e+00  +3e+03  1e-03  1e-01  7e-02  4e-02  0.5787  2e-01   1  1  1 |  0  0
 8  +2.978e+00  +3.186e+00  +2e+03  1e-03  7e-